# 2021-09-03

Trying to work out if we can deduce meltpool size, orientation, dimensions, from the images a bit better than just a size readout

Could be used to search for anomalies

In [1]:
import sys
import os
import importlib # Used during testing if I need to reload modules

import matplotlib.pyplot as plt
import numpy as np
import cv2
import scipy.stats

In [2]:
try:
    import ngif_romar.tools
except ModuleNotFoundError as error:
    # If not in path/installed, use relative import
    module_path = os.path.abspath(os.path.join(".."))
    sys.path.append(module_path)
    import ngif_romar.tools as tools

In [ ]:
dataset_path = os.path.join("..", "data", "SN2", "20200930_1500_")
logfile_path = os.path.join(dataset_path, "Data.dat")
meta_dict, data_df = tools.read_data(logfile_path)

data_df = tools.post_process_log_data(data_df)
data_df.head()

print(os.listdir(dataset_path))
frames_path = os.path.join(dataset_path, "Frames")

data_df = tools.link_camera_frames_to_df(data_df, frame_folder_path=frames_path)

data_df.head()

['Data.dat', 'Frames', 'SN2_Run1_Time_Data.txt']


Lets have a look at some images, and try to thresh

In [ ]:
subset = data_df[
    data_df["toolpath_key"] == 15
]
framename = subset["matching_frame_filename"].values[5]
frame = tools.read_and_convert_image(os.path.join(frames_path, framename))

fig, ax = plt.subplots()
imshow_result = ax.imshow(frame, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
plt.show()

# Plot without cmap to see if it's easier to see
fig, ax = plt.subplots()
imshow_result = ax.imshow(frame,  vmin=0, vmax=255)
fig.colorbar(imshow_result)
plt.show()

In [ ]:


otsu_thresh_val, thresh_img = cv2.threshold(frame, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print("Otsu thresh: value is {}".format(otsu_thresh_val))

fig, ax = plt.subplots()
imshow_result = ax.imshow(thresh_img, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
ax.set_title("Otsu thresh")
plt.show()

manual_thresh = 19
manual_thresh, thresh_img = cv2.threshold(frame, manual_thresh, 255, cv2.THRESH_BINARY)

fig, ax = plt.subplots()
imshow_result = ax.imshow(thresh_img, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
ax.set_title("Manual Thresh")
plt.show()


fig, ax = plt.subplots()
ax.hist(frame.flatten(), bins=np.arange(-0.5, 256, 1))
ax.set_title("Histogram of picture intensiteis")
ax.axvline(otsu_thresh_val, linestyle="--", label="Otsu thresh")
ax.axvline(manual_thresh, linestyle="-.", label="Manual thresh {}".format(manual_thresh))
ax.legend()
plt.show()

In [ ]:
# Lets mask off areas 

display_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

regions = [
    [0, 19],
    [19,50],
    [50, 100],
    [100, 256]
]
colours = [
    [0,0,0],
    [0,255,0],
    [0,0,255],
    [255,0,255],
]
for (low, high), colour in zip(regions, colours):
    display_img[
        (frame >= low)
        & (frame < high)
    ] = colour
    

fig, ax = plt.subplots()
ax.imshow(display_img)
plt.show()

As sort of said before, Otsu thresh is a good auto estimate, but we should look at the off frames

In [ ]:
off_frames = data_df[
    data_df["laser_off_time(ms)"] > 100
]["matching_frame_filename"].values

framename = off_frames[30]
frame = tools.read_and_convert_image(os.path.join(frames_path, framename))

fig, ax = plt.subplots()
imshow_result = ax.imshow(frame, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
ax.set_title("Frame as read in")
plt.show()

otsu_thresh_val, thresh_img = cv2.threshold(frame, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print("Otsu thresh: value is {}".format(otsu_thresh_val))

fig, ax = plt.subplots()
imshow_result = ax.imshow(thresh_img, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
ax.set_title("Otsu thresh")
plt.show()

manual_thresh = 9
manual_thresh, thresh_img = cv2.threshold(frame, manual_thresh, 255, cv2.THRESH_BINARY)

fig, ax = plt.subplots()
imshow_result = ax.imshow(thresh_img, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
ax.set_title("Manual Thresh")
plt.show()


fig, ax = plt.subplots()
ax.hist(frame.flatten(), bins=np.arange(0,256, 1))
ax.set_title("Histogram of picture intensiteis")
ax.axvline(otsu_thresh_val, linestyle="--", label="Otsu thresh")
ax.axvline(manual_thresh, linestyle="-.", label="Manual thresh {}".format(manual_thresh))
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(frame.flatten(), bins=np.arange(-0.5,np.max(frame)+1, 1))
ax.set_title("Histogram of picture intensiteis")
ax.axvline(otsu_thresh_val, linestyle="--", label="Otsu thresh {}".format(otsu_thresh_val))
ax.axvline(manual_thresh, linestyle="-.", label="Manual thresh {}".format(manual_thresh))
ax.legend()
plt.show()

Okay, looking at one or two individual frames, looking at vals > 0 is pretty good when the laser is off

We still need to figure out what's going on with the multiple peaks in the laser on frames but still

Looks like we could separate into four regions? Cold, leading edge, building area, and transition area

* Leading edge/laser area: Where the laser has just hit and the pool hasn't liquified or isn't being cooled by particles
* Building area: Pool hot/liquid, powder flowing in, building
* Transition area: between building and cold, where the laser has just started or where it's left and cooling down
* Cold: The cold areas, not necessarily as dark as the laser off images due to reflections etc

Also not that we can see the edge of the nozzle or something in the corner

Try k means clustering b/c why not?

In [ ]:
import scipy.cluster

In [ ]:
scipy.cluster.vq.kmeans

In [ ]:
subset = data_df[
    data_df["toolpath_key"] == 15
]
framename = subset["matching_frame_filename"].values[5]
frame = tools.read_and_convert_image(os.path.join(frames_path, framename))

fig, ax = plt.subplots()
imshow_result = ax.imshow(frame, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
plt.show()

# Plot without cmap to see if it's easier to see
fig, ax = plt.subplots()
imshow_result = ax.imshow(frame,  vmin=0, vmax=255)
fig.colorbar(imshow_result)
plt.show()

scipy.cluster.vq.kmeans(frame.astype(float).flatten(), 12)

In [ ]:
means, distortion = scipy.cluster.vq.kmeans(frame.astype(float).flatten(), 1)
print("Means: {}, distortion: {}".format(means, distortion))
# As a sanity check, k means with one mean should be the global mean
print(np.mean(frame))

Distortion is mean Euclidean distance between observations as entered and their centroids

Basically, lower means points close to value

See https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.kmeans.html

In [ ]:
distortions = []
ks = np.arange(2, 10, 1)
for k in ks:
    means, distortion = scipy.cluster.vq.kmeans(frame.astype(float).flatten(), k)
    distortions.append(distortion)

fig, ax = plt.subplots()
ax.scatter(ks, distortions)
ax.set_xlabel("K (number means)")
ax.set_ylabel("Distortion")
plt.show()

Turns around 5 or 6 means, which we sort of saw before

In [ ]:
means, distortion = scipy.cluster.vq.kmeans(frame.astype(float).flatten(), 4)
means

In [ ]:
subset = data_df[
    data_df['laser_on_time(ms)'] > 100
]
framename = subset["matching_frame_filename"].values[275]
frame = tools.read_and_convert_image(os.path.join(frames_path, framename))

fig, ax = plt.subplots()
imshow_result = ax.imshow(frame, cmap='gray', vmin=0, vmax=255)
fig.colorbar(imshow_result)
plt.show()

# Plot without cmap to see if it's easier to see
fig, ax = plt.subplots()
imshow_result = ax.imshow(frame,  vmin=0, vmax=255)
fig.colorbar(imshow_result)
plt.show()



In [ ]:
means, distortion = scipy.cluster.vq.kmeans(frame.astype(float).flatten(), 4)
means, distortion

In [ ]:
means, distortion = scipy.cluster.vq.kmeans(frame.astype(float).flatten(), 4)
means, distortion

display_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
means = np.sort(means)
colours = [
    [0,0,0],
    [255,0,0],
    [0,255,0],
    [0,0,255],
    [255,0,255],
    [0, 255, 255],
]

for i in range(0, frame.shape[0]):
    for j in range(0, frame.shape[1]):
        display_img[i,j] = colours[np.argmin(np.abs(means - frame[i,j]))]
        
fig, ax = plt.subplots()
ax.imshow(display_img)
plt.show()
print(means)

Going by the above on a couple of images, I don't think I'm fair off with my 4 region idea

If we just look at the onnish ones, its a thresh of mean_1 + (mean_2 - mean_1)/2 ~ whatever

In [ ]:
thresh_val = means[0] + (means[0] + means[1]) / 2
print(thresh_val)
# thresh_val=29

fig, ax = plt.subplots()
ax.hist(frame.flatten(), bins=np.arange(-0.5, 256, 1))
ax.axvline(thresh_val, linestyle="--")
plt.show()

thresh_val, thresh_img = cv2.threshold(frame, thresh_val, 255, cv2.THRESH_BINARY)
fig, ax = plt.subplots()
ax.imshow(thresh_img, cmap='gray')
plt.show()

In [ ]:
# Lets go through and sample a bunch of images, and check histo

subset = data_df[
    data_df['laser_on_time(ms)'] > 100
]
framenames = subset["matching_frame_filename"]
print("Total number of frames to select: {}".format(len(framenames)))
sample_size = 150
sample_framenames = np.random.choice(framenames, sample_size, replace=False)
all_pixels = []
for framename in sample_framenames:
    frame = tools.read_and_convert_image(os.path.join(frames_path, framename))
    all_pixels = np.concatenate([all_pixels, frame.flatten()])

In [ ]:
fig, ax = plt.subplots()
ax.hist(all_pixels, bins=np.arange(-0.5, 256, 1))
plt.show()

Still need to figure out a consistent but per image threshold, and then we can look at the orientation and directions

In [ ]:
# First pass, just do the Otsu thresh, even if we have problems with it

thresh_val, thresh_img = cv2.threshold(frame, 40, 255, cv2.THRESH_BINARY)
print(thresh_val)

fig, ax = plt.subplots()
ax.hist(frame.flatten(), bins=np.arange(-0.5, 256, 1))
plt.show()

fig, ax = plt.subplots()
ax.imshow(thresh_img, cmap='gray')
plt.show()

In [ ]:
# Go to threshed image and do connected components, find largest
retval, labels, stats, centroid = cv2.connectedComponentsWithStats(thresh_img)
# Stats is an array, row for each connected component, stats are tl x, tl y, bbox width, bbox height, area

display_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
for label in range(0, retval):
    display_img[labels==label] = np.random.randint(0,256, size=3)

fig, ax = plt.subplots()
ax.imshow(display_img)
plt.show()

display_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
display_img = display_img * 0
max_size_label = np.argmax(stats[1:,4]) +1 # Plus one because we offset the array to not inc background
display_img[labels == max_size_label] = [255,255,255]

# Print just the largest
fig, ax = plt.subplots()
ax.imshow(display_img)
plt.show()

## So far:

What we've done is have a look at putting into three groups via knn method. It seems to line up with the basic idea of roughly four temperature levels

We also had a quick look at the Otsu thresh; it's not perfect, but works sometimes. With that we can get a different blob.

## Next:

Lets form a method that can measure the dimensions of the shape

The camera should be calibrated against a curve, so we should be able to set the threshold as a set temp. We can always refine this. In previous sections we combined multiple images to try and see if there were clear peaks

Note that for the moment we're mainly concerned with width

### Ideas:
* Look at the outer regions of the image; we know that this area should be cold and empty so we can use that as a thresh
* See if we can consistently find the orientation of the image, then align to that and do row and col scans

### Using background to find thresh

Open a bunch of images, mask out centre, use that to find background level of pixels

In [ ]:
subset = data_df[
    data_df['laser_on_time(ms)'] > 100
]
framenames = subset["matching_frame_filename"]
print("Total number of frames to select: {}".format(len(framenames)))

# Choosing arb image
framename = framenames[100]

frame = tools.read_and_convert_image(os.path.join(frames_path, framename))

fig, ax = plt.subplots()
imshow_result = ax.imshow(frame)
fig.colorbar(imshow_result)
plt.show()

# Create a mask
ii, jj = np.indices(frame.shape)
# Centring coords
ii = ii - frame.shape[0]//2
jj = jj - frame.shape[1]//2
# Will be used for mask; at the moment an array of distance from centre
distances = np.linalg.norm([ii,jj], axis=0)
mask = np.zeros_like(frame)
mask[
    (distances > 55)
    & (distances < 105)
    & (ii < 0) #Getting upper part, chosen image has a tail
] = 1

masked_in = frame*mask
masked_out = frame*(~mask//255)

fig, ax = plt.subplots()
imshow_result = ax.imshow(mask, cmap='gray')
fig.colorbar(imshow_result)
ax.set_title("Proposed mask")
plt.show()

fig, ax = plt.subplots()
imshow_result = ax.imshow(~mask, cmap='gray')
fig.colorbar(imshow_result)
ax.set_title("Inv. of mask")
plt.show()

fig, ax = plt.subplots()
imshow_result = ax.imshow(masked_in)
fig.colorbar(imshow_result)
ax.set_title("Masked in")
plt.show()

fig, ax = plt.subplots()
imshow_result = ax.imshow(masked_out)
fig.colorbar(imshow_result)
ax.set_title("Masked out")
plt.show()

In [ ]:
# Not including the zero parts buecause it blows out the scale
fig, ax = plt.subplots()
bins = np.arange(0.5, 20, 1)
ax.hist(masked_in.flatten(), bins=bins, density=True)
xs = np.linspace(0, 20, 100)
ax.plot(xs+2.1, scipy.stats.gamma.pdf(xs, 4))
# Putting in gamma distro for comp.
ax.set_title("Masked in; dark parts")
plt.show()

print("Masked in: mean {:.3g}, std {:.3g}, sqrt(mean) {:.3g}".format(
    np.mean(masked_in), np.std(masked_in), np.sqrt(np.mean(masked_in))
))

fig, ax = plt.subplots()
ax.hist(masked_out.flatten(), bins=np.arange(0.5, 256, 1))
ax.set_title("Masked out; outer parts")
plt.show()

The outer parts look kind of poissonian/gamma distro, which is interesting

In [ ]:
subset = data_df[
    data_df['laser_on_time(ms)'] > 100
]
framenames = subset["matching_frame_filename"]
print("Total number of frames to select: {}".format(len(framenames)))

# Choosing arb image
framename = framenames[9000]

frame = tools.read_and_convert_image(os.path.join(frames_path, framename))

thresh_val = 20
thresh_val, thresh_img = cv2.threshold(frame, thresh_val, 255, cv2.THRESH_BINARY)
# thresh_val, thresh_img = cv2.threshold(frame, 40, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print(thresh_val)

fig, ax = plt.subplots()
imshow_result = ax.imshow(frame)
fig.colorbar(imshow_result)
ax.set_title("Input image")
plt.show()

# Look at rows, roughly centre +/- a bit
row_nums = [
    80, 60, 100
]
fig, ax = plt.subplots()
for i in row_nums:
    ax.plot(frame[i, :], label="Row {}".format(i))
ax.legend()
ax.set_title("Row scan of pixel values")
plt.show()

# Look at cols as well
col_nums = [
    150, 120, 90
]
fig, ax = plt.subplots()
for j in col_nums:
    ax.plot(frame[:, j], label="Col {}".format(j))
ax.legend()
ax.set_title("Column scan of pixel values")
plt.show()


fig, ax = plt.subplots()
ax.hist(frame.flatten(), bins=np.arange(-0.5, 256, 1))
ax.axvline(thresh_val)
ax.set_title("Histogram of image valuse")
plt.show()

fig, ax = plt.subplots()
ax.imshow(thresh_img, cmap='gray')
ax.set_title("Threshed image")
plt.show()

retval, labels, stats, centroid = cv2.connectedComponentsWithStats(thresh_img)
# Stats is an array, row for each connected component, stats are tl x, tl y, bbox width, bbox height, area

display_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
for label in range(0, retval):
    display_img[labels==label] = np.random.randint(0,256, size=3)

fig, ax = plt.subplots()
ax.imshow(display_img)
plt.show()

display_img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
display_img = display_img * 0
max_size_label = np.argmax(stats[1:,4]) +1 # Plus one because we offset the array to not inc background
display_img[labels == max_size_label] = [255,255,255]




# Now lets grab a uint8 image to work on further
proc_image = np.zeros_like(thresh_img)
proc_image[labels == max_size_label] = 255


# Do a close to kill islands
# Do a morpho close transform
kernel = np.ones((10,10), np.uint8)
proc_image = cv2.morphologyEx(proc_image, cv2.MORPH_CLOSE, kernel)

fig, ax = plt.subplots()
imshow_result = ax.imshow(proc_image, cmap='gray')
fig.colorbar(imshow_result)
plt.show()

Now we've processed the image etc we have a blob to work with

We want to fit a vague ellipse, and measure dimensions

In [ ]:
edges = cv2.Canny(proc_image, 100, 100,)


fig, ax = plt.subplots()
imshow_result = ax.imshow(edges, cmap='gray')
fig.colorbar(imshow_result)
plt.show()

# Draw the edges onto the original

Use that old PCA method I guess

In [ ]:
ii, jj = np.indices(proc_image.shape)
ii = ii[proc_image > 0] 
jj = jj[proc_image > 0]

coords = np.vstack([ii, jj]).transpose()
# Now have an array of (i,j) correspond to the blob

# PCA: now centre at mean
coords = coords - np.mean(coords)
# Covariance matrix

# Remember that diagonals are variance of values
cov_mat = np.cov(coords, rowvar=False)
# Then if we diag the covariance matrix we should get an estimate of extent and orientation from eigenvectors
eig_vals, eig_vectors = np.linalg.eig(cov_mat)


In [ ]:
angle = -1 * np.arctan2(eig_vectors[1,0], eig_vectors[0,0]) # align to have first eig vector on X axis

rot_mat = np.array([
    [np.cos(angle), -np.sin(angle)],
    [np.sin(angle), np.cos(angle)],
])

rot_mat @ eig_vectors

In [ ]:
# Now rotate all coords, and then our eigenvectors corresp. to highest variance will be aligned

rot_coords = np.array([rot_mat @ coord for coord in coords])
rot_cov = np.cov(rot_coords, rowvar=False)
np.sqrt(rot_cov[0,0]), np.sqrt(rot_cov[1,1])

In [ ]:
# Look at rotated image

centre =  (frame.shape[0]//2, frame.shape[1]//2)
transform_mat = cv2.getRotationMatrix2D(centre, angle, 1)

rot_image = cv2.warpAffine(frame, transform_mat, frame.shape[::-1])

fig, ax = plt.subplots()
ax.imshow(rot_image)
plt.show()

So, we can use a PCA type method to find the orientation and rotate to that. Image rotation might not be that helpful, but still

## Another idea: Line scan from middle of image

We know that the middle of the image should be roughly coincident with the laser, so let's scan in lines around that, and check width

In [ ]:
def interpolate_pixels_along_line(x0, y0, x1, y1):
    """
    From https://stackoverflow.com/questions/24702868/python3-pillow-get-all-pixels-on-a-line
    Not ideal at the moment, need to make check that we're getting appropriate samples along lines but still
    
    Uses Xiaolin Wu's line algorithm to interpolate all of the pixels along a
    straight line, given two points (x0, y0) and (x1, y1)

    Wikipedia article containing pseudo code that function was based off of:
        http://en.wikipedia.org/wiki/Xiaolin_Wu's_line_algorithm
    """
    pixels = []
    steep = abs(y1 - y0) > abs(x1 - x0)

    # Ensure that the path to be interpolated is shallow and from left to right
    if steep:
        t = x0
        x0 = y0
        y0 = t

        t = x1
        x1 = y1
        y1 = t

    if x0 > x1:
        t = x0
        x0 = x1
        x1 = t

        t = y0
        y0 = y1
        y1 = t

    dx = x1 - x0
    dy = y1 - y0
    gradient = dy / dx  # slope

    # Get the first given coordinate and add it to the return list
    x_end = round(x0)
    y_end = y0 + (gradient * (x_end - x0))
    xpxl0 = x_end
    ypxl0 = round(y_end)
    if steep:
        pixels.extend([(ypxl0, xpxl0), (ypxl0 + 1, xpxl0)])
    else:
        pixels.extend([(xpxl0, ypxl0), (xpxl0, ypxl0 + 1)])

    interpolated_y = y_end + gradient

    # Get the second given coordinate to give the main loop a range
    x_end = round(x1)
    y_end = y1 + (gradient * (x_end - x1))
    xpxl1 = x_end
    ypxl1 = round(y_end)

    # Loop between the first x coordinate and the second x coordinate, interpolating the y coordinates
    for x in range(xpxl0 + 1, xpxl1):
        if steep:
            pixels.extend([(np.floor(interpolated_y), x), (np.floor(interpolated_y) + 1, x)])

        else:
            pixels.extend([(x, np.floor(interpolated_y)), (x, np.floor(interpolated_y) + 1)])

        interpolated_y += gradient

    # Add the second given coordinate to the given list
    if steep:
        pixels.extend([(ypxl1, xpxl1), (ypxl1 + 1, xpxl1)])
    else:
        pixels.extend([(xpxl1, ypxl1), (xpxl1, ypxl1 + 1)])

    return pixels

In [ ]:
display_image = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

angles = np.linspace(-np.pi/2, np.pi/2, 10)
angles = angles[:-1]

centre = np.array([frame.shape[0]//2, frame.shape[1]//2])
max_dim = 256

pixel_values = []

for angle in angles:
    # Start, end in i,j
    start = centre - max_dim * np.array([np.sin(angle), np.cos(angle)])
    end = centre + max_dim * np.array([np.sin(angle), np.cos(angle)])
    start = start.astype(int)
    end = end.astype(int)
    # Line drawing takes xy
    cv2.line(display_image, tuple(start[::-1]), tuple(end[::-1]), (255,0,0), 1)
    
    # Now try and get pixel values
    indices = np.array(interpolate_pixels_along_line(*start, *end))
    # Ensure we only get pts inside image
    indices = indices[
        (indices[:,0] >= 0)
        & (indices[:, 1] >= 0)
        & (indices[:, 0] < frame.shape[0])
        & (indices[:,1] < frame.shape[1])
    ]
    # Round
    indices = np.round(indices).astype(int)
    these_pixel_values = frame[indices[:,0], indices[:,1]]
    pixel_values.append(these_pixel_values)
    
    
    

fig, ax = plt.subplots()
ax.imshow(display_image)
plt.show()

In [ ]:

for line_sample, angle in zip(pixel_values, angles):
    fig, ax = plt.subplots()
    ax.plot(line_sample)
    ax.set_title("Angle: {}/{}".format(angle, np.degrees(angle)))
plt.show()